In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go


from credentials import plotly_token  

import sqlite3

import dash
import dash.dependencies as dd
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

import pandas as pd
import numpy as np


from io import BytesIO
from PIL import Image
from wordcloud import WordCloud
import base64

import nltk
from nltk.probability import FreqDist
from collections import deque
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amudhini\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Amudhini\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
df=pd.read_csv('etweets.csv')

app = dash.Dash(external_stylesheets=[dbc.themes.LUX])

app.index_string = '''
<!DOCTYPE html>
<html>
    {%app_entry%}
    {%config%}
    {%css%}
    {%scripts%}
    {%renderer%}
    <body style="background:black !important;">
    </body>
</html>
'''


app.layout = html.Div([
            dcc.Location(id="url"),
            dbc.NavbarSimple(
                children=[
                    dbc.NavLink("Home", href="/home", id="page-1-link"),
                    dbc.NavLink("Analysis by State", href="/analysis-by-state", id="page-2-link"),
                    dbc.NavLink("Analysis by Keyword", href="/analysis-by-keyword", id="page-3-link"),
                    dbc.NavLink("Headlines", href="/headlines",id="page-4-link"),
                    dbc.NavLink("About",href="/about",id="page-5-link")],
                brand="Sentiment Analysis of COVID-19 Tweets",
                className="navbar navbar-expand-lg navbar-dark bg-dark"),
            html.Div(id="page-content",style={'margin-right': '3%','margin-left':'3%',
                                              'margin-top':'2%','margin-bottom':'2%'}),
        ],style={'background':'black','height':'100%'})


def update_file():
    global df
    df=pd.read_csv('etweets.csv')
    
    
# This callback uses the current pathname to set the active state of the
# corresponding nav link to true, allowing users to tell see page they are on
@app.callback([dd.Output(f"page-{i}-link", "active") for i in range(1, 6)],
              [dd.Input("url", "pathname")])
def toggle_active_links(pathname):
    if pathname == "/":
        # Treat page 1 as the homepage / index
        return True, False, False, False, False
    return [pathname == f"/page-{i}" for i in range(1, 6)]


@app.callback(dd.Output("page-content", "children"), 
              [dd.Input("url", "pathname")])
def render_page_content(pathname):
    if pathname in ["/", "/home"]:
        home= [
            dbc.Card([dbc.CardHeader(["Live sentiment line plot"]),
                      html.Div([dcc.Graph(id='live-update-graph'),
                                dcc.Interval(id='interval-component',interval=2*1000, n_intervals=0)],
                               className='rounded')],
                     style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'}
                    ,className="card text-white bg-primary"),
            
            html.Div([
                html.Div(children=[
                    html.Div(
                        dbc.Card([dbc.CardHeader(["Sentiments around the world"]),dcc.Graph(id='tweets-world')],
                                className="card text-white bg-primary"),
                        style={'width': '40%'},className='d-inline-block rounded align-self-center'),
                    
                    html.Div([dbc.Card([
                        dbc.CardHeader([
                            html.Div(children=["Geographic distribution"],
                                     className='card text-white bg-primary d-inline-block align-self-center',
                                     style={'width': '120%'}),
                            dcc.Dropdown(id='tweet-choice',
                                         options=[{'label': 'World', 'value': 'world'},
                                                  {'label': 'India', 'value': 'IN'}], value='IN',
                                         className='d-inline-block',style={'width': '120%','color':'black'})],
                        className='d-flex justify-content-between'),
                        
                        dcc.Graph(id='tweets-map')],className="card text-white bg-primary")],
                             style={'width': '50%'},className='d-inline-block rounded align-middle'),
                
                ],className='d-flex justify-content-around')],
                style={'margin-top':'2%','margin-bottom':'2%'}),
            
            
            html.Div([
                html.Div(children=[
                    dbc.Card([dbc.CardHeader(["Pie-chart"]),
                              dcc.Graph(id='sentiments-pie-chart')],className="card text-white bg-primary")],
                         style={'width': '40%'},className='h-50 d-inline-block rounded'),
                
                html.Div(children=[
                    dbc.Card([dbc.CardHeader(["Wordcloud"]),
                              html.Img(id="overallsentiment_wc")],className="card text-white bg-primary")],
                         style={'width': '50%'},className='h-50 d-inline-block rounded align-middle')
            ],style={'margin-top':'2%','margin-bottom':'2%'},
            className='d-flex justify-content-around')]
        
        return home
    
    elif pathname == "/analysis-by-state":
        states=[
            dbc.Card([dbc.CardHeader(["Sentiments in different states"]),
                      html.Div(dcc.Graph(id='Figure1'))],
                     style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
                    className="card text-white bg-primary"),
                                                                                                                       
            
            dbc.Card([dbc.CardHeader(["Analysis of a state"]),
                      dbc.CardBody(style={'background':'black'},children=[
                          html.Div(dcc.Dropdown(id='group-select',value='Maharashtra'),
                                   style={"display":'block',"width":'30%',"color":"black","font-size":"20px",
                                          "border-style":"groove","float":"center",'text-align': 'center',
                                          'margin-left':'auto','margin-right': 'auto','vertical-align':'middle'}),
                          
                          html.Div(dcc.Graph(id="graph"),
                                   style={"display":"block","width":"80%","margin-top":"2%" ,
                                          'margin-left':'auto','margin-right': 'auto','vertical-align':'middle'}), 
                          
                          html.Div(dcc.Graph(id="graph1"),
                                   style={"width":"40%","margin-top":"2%" ,'margin-left':'auto',
                                          'margin-right': 'auto','vertical-align':'middle'}),])
                      ],style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
                     className="card text-white bg-primary"
                    )]
    
        return states
    
    elif pathname == "/analysis-by-keyword":
        keyword=[
            dbc.Card([dbc.CardHeader(["Top keywords in India"]),
                      html.Div(dcc.Graph(id='polar_bar_graph'))],
                     className="card text-white bg-primary",
                      style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'}),
            
            dbc.Card([dbc.CardHeader(["Analysis of a keyword"]),
                      dbc.CardBody(style={'background':'black'},children=[
                      html.Div([
                          dbc.FormGroup([
                          dbc.Label("Enter the keyword",style={"color":"white","font-size":"125%","width":"20%"}),
                          dbc.Input(id="word_input",value="corona",type="text")]),
                          dcc.Graph(id="Graph")]
                      ,style={'margin-right':'auto','margin-left':'auto','width':'80%'})])],
                    style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
                    className="card text-white bg-primary"),
                      
                      
            dbc.Card([dbc.CardHeader(["Wordcloud of each sentiment"]),
                      dbc.CardBody(style={'background':'black'},children=[
                          html.Div([
                              html.Img(id="positive_wc",style={'width': '25%'},
                                       className="h-25 d-inline-block border border-secondary rounded-circle"),
                              html.Img(id="neutral_wc",style={'width': '25%'},
                                       className="h-25 d-inline-block border border-secondary rounded-circle"),
                              html.Img(id="negative_wc",style={'width': '25%'},
                                       className="h-25 d-inline-block border border-secondary rounded-circle")
                    ],id="wordcloud",className='d-flex justify-content-around')])],
                    style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
                    className="card text-white bg-primary")
            
            ]
        
        return keyword
    
    elif pathname == "/headlines":
        
        twitter=[dbc.Container([        
            html.Iframe(
                height="1000px", width="100%",
                srcDoc='''
                <rssapp-wall id="yj6FFMWLIZLGWjUp"></rssapp-wall>
                <script src="https://widget.rss.app/v1/wall.js" type="text/javascript" async></script>
                '''
            ),
            dbc.Row([dbc.Col(html.Div([
                html.Iframe(
                    width="100%", height='540px',
                    srcDoc='''
                    <a class="twitter-timeline" data-lang="en" data-theme="dark" 
                    href="https://twitter.com/WHO?ref_src=twsrc%5Etfw">Tweets by WHO</a> 
                    <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
                    '''
                )
            ]),lg=4, md=4, xs=12),
                     dbc.Col(html.Div([
                         html.Iframe(
                             width="100%", height='540px',
                             srcDoc='''
                             <a class="twitter-timeline" data-lang="en"  data-theme="dark" 
                             href="https://twitter.com/MoHFW_INDIA?ref_src=twsrc%5Etfw">Tweets by MoHFW_INDIA</a>
                             <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
                             '''
                         )
                     ]),lg=4, md=4, xs=12),
                     dbc.Col(html.Div([
                         html.Iframe(
                             width="100%",
                             height='540px',
                             srcDoc='''
                             <a class="twitter-timeline" data-lang="en"  data-theme="dark" 
                             href="https://twitter.com/PTI_News?ref_src=twsrc%5Etfw">Tweets by PTI_News</a> 
                             <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
                             '''
                         )
                     ]),lg=4, md=4, xs=12)
                    ],style={'margin-top':'2%'})
        ])]
                                         
        return twitter
        
    elif pathname == "/about":
        about=[
            dbc.Card(
                dbc.CardBody([
                    html.H4("About our project", className="card-title"),
                    html.P(
                        "The Coronavirus pandemic has brought the world to a standstill. "
                        "The media highlights various issues caused due to the pandemic on the economic dip, "
                        "the spread of the influenza and about the measures taken by the government to stop "
                        "the spread of the influenza. The media forgets to highlight the emotions of the people "
                        "during this lockdown while streaming the information to the people as a result of "
                        "which people don’t get to look into the emotions of their fellow beings. ",className="card-text"),
                    html.P(
                        "Our project brings a solution to this problem. "
                        "Our project analyses the tweets posted by the users in their respective twitter handle "
                        "and analyses the sentiment of people during this pandemic. "
                        "Features like line chart, geographical mapping of the sentiments, search by keyword "
                        "and search by state allow the user to learn about the emotions of the people. "
                        "The line chart and geographical mapping will help the user get a clear picture of the "
                        "emotions of people on a larger scale. Search by keyword and search by state will "
                        "allow the user to explore into the intricate details and will allow them to "
                        "understand the usage of a word in different localities and allows the user to find "
                        "out the emotions of a particular state or community respectively. ",className="card-text"),
                    html.P(
                        "Let's stand together in these tough times by practising social distancing and "
                        "come out of this situation strong. Be strong and stay safe.",
                        className="card-text",)
                ]),
                style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
            className="card text-white bg-primary"),
            
            dbc.Card(
                dbc.CardBody([
                    html.H4("Project is created by",className="card-title"),
                    html.P("V. Maheysh",className="card-text"),
                    html.P("P. K. Amudhini",className="card-text"),
                    html.P("S. Divyashree",className="card-text"),
                    html.P("A. Shalini",className="card-text"),
                ]),
                style={'margin-right': '2%','margin-left':'2%','margin-top':'2%','margin-bottom':'2%'},
            className="card text-white bg-primary")
        ]
        
        return about
        
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )



'''------------------------------------ HOME ------------------------------------'''

X = deque(maxlen=20)
X.append(1)
Y = deque(maxlen=20)
Y.append(1)
MAX_DF_LENGTH = 100
def df_resample_sizes(df, maxlen=MAX_DF_LENGTH):
    df_len = len(df)
    resample_amt = 100
    vol_df = df.copy()
    vol_df['volume'] = 1

    ms_span = (df.index[-1] - df.index[0]).seconds * 1000
    rs = int(ms_span / maxlen)

    df = df.resample('{}ms'.format(int(rs))).mean()
    df.dropna(inplace=True)

    vol_df = vol_df.resample('{}ms'.format(int(rs))).sum()
    vol_df.dropna(inplace=True)

    df = df.join(vol_df['volume'])

    return df

@app.callback(dd.Output('live-update-graph', 'figure'),[dd.Input('interval-component', 'n_intervals')])
def update_graph_live(n):
    update_file()
    conn = sqlite3.connect('twitter.db')
    c = conn.cursor()
    df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE '%covid%' ORDER BY unix DESC LIMIT 1000", conn)
    df.sort_values('unix', inplace=True)
    
    df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/5)).mean()
    df['date'] = pd.to_datetime(df['unix'],unit='ms')
    df.set_index('date', inplace=True)
    #df = df.resample('1min').mean()#####for more curved graphs while streaming
    df = df_resample_sizes(df)
    df.dropna(inplace=True)
   
    X = df.index
    Y = df.sentiment_smoothed
    

    data = go.Scatter(x=X, y=Y, name='Scatter', mode= 'lines+markers')

    return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)],title='Time'),
                                                yaxis=dict(range=[min(Y),max(Y)],title='Sentiment'),
                                                font={'color':'#FFFFFF'},
                                                plot_bgcolor = '#0C0F0A',
                                               paper_bgcolor ='#0C0F0A', )}

    

@app.callback(dd.Output('tweets-map', 'figure'), [dd.Input('tweet-choice', 'value')])
def return_fig_map(map_choice):
    if map_choice=='IN':
        df1=df[df['Country_code']=='IN']
        z=3
    else:
        df1=df
        z=1
    
    df1=df1[-1000:]
    fig = px.scatter_mapbox(df1, lat="Latitude", lon="Longitude", color="Overall_sentiment",
                            hover_data=["Sentiment_compound"],zoom=z,template='plotly_dark',
                            color_discrete_map={'Positive':'rgb(50, 205, 50)',
                                                'Negative':'rgb(255, 51, 51)',
                                                'Neutral':'rgb(176, 196, 222)'})

    fig.update_layout(mapbox_style="dark", mapbox_accesstoken=plotly_token)
    fig.update_traces(hovertemplate='Latitude: %{lat} <br>Longitude: %{lon}')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig


@app.callback(dd.Output('tweets-world', 'figure'), [dd.Input('tweets-world', 'id')])
def return_fig_world(id):
    df1=df[-2000:]
    data=[go.Scattergeo(
            lat=df1[df1['Overall_sentiment']=='Positive']['Latitude'],
            lon=df1[df1['Overall_sentiment']=='Positive']['Longitude'],
            mode='markers',
            marker_color='rgb(50, 205, 50)',
            marker_size=3,
            name='Positive'),
     go.Scattergeo(
            lat=df1[df1['Overall_sentiment']=='Negative']['Latitude'],
            lon=df1[df1['Overall_sentiment']=='Negative']['Longitude'],
            mode='markers',
            marker_color='rgb(255, 51, 51)',
            marker_size=3,
            name='Negative'),
    go.Scattergeo(
            lat=df1[df1['Overall_sentiment']=='Neutral']['Latitude'],
            lon=df1[df1['Overall_sentiment']=='Neutral']['Longitude'],
            mode='markers',
            marker_color='rgb(176, 196, 222)',
            marker_size=3,
            name='Neutral')]

    layout =go.Layout(template='plotly_dark',
        geo=go.layout.Geo(
            showland = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = 'rgb(51, 102, 0)',
            oceancolor = 'rgb(0, 0, 102)',
            projection_type='orthographic',
            center_lon=50,
            center_lat=0,
            projection_rotation_lon=50
        ))
    lon_range = np.arange(-180, 180, 2)

    frames = [go.Frame(layout=go.Layout(geo_center_lon=lon,geo_projection_rotation_lon =lon,
                                        geo_center_lat=0,geo_projection_rotation_lat=0),
                   name =f'{k+1}') for k, lon in enumerate(lon_range)]


    sliders = [dict(steps = [dict(method= 'animate',args= [[f'{k+1}'], 
                                          dict(mode= 'immediate',
                                          frame= dict(duration=0, redraw= True),
                                          transition=dict(duration= 0))],label=f'{k+1}') for k in range(len(lon_range))], 
                transition= dict(duration= 0 ),
                x=0, # slider starting position  
                y=0,   
                len=1.0) #slider length
              ]

    fig = go.Figure(data=data, layout=layout, frames=frames)
    fig.update_layout(sliders=sliders)
    
    return fig

@app.callback(dd.Output('sentiments-pie-chart', 'figure'), [dd.Input('sentiments-pie-chart', 'id')])
def return_pie_chart(id):
    df3=df['Overall_sentiment'].value_counts()
    df3=df3.rename_axis('index').reset_index()
    fig = go.Figure(data=[go.Pie(labels=['Positive','Negative','Neutral'], values=df3.Overall_sentiment, hole=.6)])
    fig.update_traces(hoverinfo='label+percent', textinfo='label', textfont_size=20,
                  marker=dict(colors=['rgb(34, 139, 34)', 'rgb(255, 69, 0)', 'rgb(65, 105, 225)'],
                              line=dict(color='rgb(245, 255, 250)', width=2)))
    fig.update_layout(template='plotly_dark',height=400)
    
    return fig


@app.callback(dd.Output('overallsentiment_wc', 'src'),[dd.Input('overallsentiment_wc', 'id')])
def return_image(id):
    img = BytesIO()
    fd =return_tweet_words(df)
    d= pd.DataFrame({'words': list(fd.keys()),'Count' : list(fd.values())})
    d= d.nlargest(columns = 'Count', n = 70)
    mask1=np.array(Image.open("photos\\mask.png"))
    plot_wordcloud(data=d,mask=mask1,contour_color='white').save(img, format='PNG')
    
    return 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())



'''------------------------------------ SEARCH BY STATE ------------------------------------'''

@app.callback(dd.Output("Figure1",'figure'),[dd.Input("Figure1",'id')])
def graph(id):
    df1=df[df['Country_code']=='IN']
    
    a=pd.DataFrame()
    a['State']=df1['State'].unique()
    a['total_count']=a['State'].apply(lambda x: len(df1[df1['State']==x]))
    a['positive_count']=a['State'].apply(lambda x: len(df1[df1['State']==x]['State'][df1[df1['State']==x]['Overall_sentiment']=='Positive']))
    a['negative_count']=a['State'].apply(lambda x: len(df1[df1['State']==x]['State'][df1[df1['State']==x]['Overall_sentiment']=='Negative']))
    a['neutral_count']=a['State'].apply(lambda x: len(df1[df1['State']==x]['State'][df1[df1['State']==x]['Overall_sentiment']=='Neutral']))
    a.sort_values(by=['total_count'], inplace=True,ascending=False)

    trace3 = go.Bar(x =a.State.head(10), y = a.positive_count.head(10), name='positive')
    trace2 = go.Bar(x= a.State.head(10), y = a.negative_count.head(10), name ='negative')
    trace1 = go.Bar(x = a.State.head(10), y = a.neutral_count.head(10), name ='neutral' )
    data = [trace1, trace2, trace3]
    layout = go.Layout(barmode = 'group',paper_bgcolor="Black",
                       font=dict(family="Nunito Sans",size=15,color="#DDDDDD"))
    fig= go.Figure(data = data, layout = layout)
    
    return(fig)


@app.callback(dd.Output("group-select",'options'),[dd.Input("group-select",'id')])
def preprocessing(id):
    df1=df[df['Country_code']=='IN']
    p=df1.groupby('State').count()
    p.sort_values('User_Id',inplace=True,ascending=False)
    options_list=[];
    for j in p.index:
        options_list.append({'label':j,'value':j})
        
    return options_list


@app.callback([dd.Output("graph",'figure'),dd.Output("graph1",'figure')],[dd.Input('group-select','value')])
def update_output_div(m):
    df1=df[df['State']==m]

    f=return_tweet_words(df1)
    sort_orders = sorted(f.items(), key=lambda x: x[1], reverse=True)
    x=[];y=[]
    for j in range(0,10):
        x.append(sort_orders[j][0])
        y.append(sort_orders[j][1])

    y1=[];y2=[];y3=[];x1=[]

    for k in range(0,7):
        p=0;n=0;l=0
        for j in df1.index:
            if df1['Overall_sentiment'][j]=='Positive' and str(df1['Tweet_text'][j]).lower().count(x[k])>0:
                p+=str(df1['Tweet_text'][j]).lower().count(x[k])
            elif df1['Overall_sentiment'][j]=='Negative'and str(df1['Tweet_text'][j]).lower().count(x[k])>0:
                n+=str(df1['Tweet_text'][j]).lower().count(x[k])
            elif df1['Overall_sentiment'][j]=='Neutral'and str(df1['Tweet_text'][j]).lower().count(x[k])>0:
                l+=str(df1['Tweet_text'][j]).lower().count(x[k])      
        if(p>0 or n>0 or l>0):
            y1.append(p)
            y2.append(n)
            y3.append(l)
            x1.append(x[k])

    fig = go.Figure(go.Bar(x=x1, y=y3, name='Neutral'))
    fig.add_trace(go.Bar(x=x1, y=y2, name='Negative'))
    fig.add_trace(go.Bar(x=x1, y=y1, name='Positive'))
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis_title="Frequent Words Used", yaxis_title="Count",
                      font=dict(family="Nunito Sans", size=15, color="#DDDDDD"),
                       paper_bgcolor="Black")

        
    p=len(df1[df1['Overall_sentiment']=='Positive'])
    n=len(df1[df1['Overall_sentiment']=='Negative'])
    l=len(df1[df1['Overall_sentiment']=='Neutral'])
    
    
    fig1 = go.Figure(data=go.Scatterpolar(r=[p,n,l],theta=['Positive','Negative','Neutral'],fill='toself',
                                         hovertemplate = 'Sentiment: %{theta}<br>Count: %{r}<extra></extra>'))
    fig1.update_layout(paper_bgcolor="Black",font=dict(family="Nunito Sans",size=15,color="#DDDDDD"),
                       polar=dict(bgcolor='#1e2130',radialaxis=dict(visible=True,)),
                       showlegend=False)
    fig1.update_layout(title_text='Sentiment polarity of the state', title_x=0.5)
    
    return fig,fig1


'''------------------------------------ SEARCH BY KEYWORD ------------------------------------'''

def return_tweet_words(df):
    words = set(nltk.corpus.words.words())
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words+=['https','http','www','www.','like','still','every','get',
                 'made','this','can','they','amp','yet','may','also']

    tweet_words=[]
    for line in list(df["Tweet_text"]):
        for word in str(line).split():
            tweet_words.append(word.lower())
    
    tw=[w for w in tweet_words if not w in stop_words and len(w)>2 and not(w.isnumeric())]
    fdist=FreqDist(tw)
    
    return fdist


def plot_wordcloud(data,mask,contour_color):
    d = {a: x for a, x in data.values}
    wc = WordCloud(mask=mask,contour_width=3,contour_color=contour_color,background_color='black', width=480, height=360)
    wc.fit_words(d)
    
    return wc.to_image()


@app.callback([dd.Output('positive_wc', 'src'),dd.Output('neutral_wc', 'src'),dd.Output('negative_wc', 'src')], 
              [dd.Input('wordcloud', 'id')])
def make_image(b):
    df1=df[df["Country_code"]=='IN']
    
    img1 = BytesIO()
    df_p=df1[df1['Overall_sentiment'] =='Positive']
    fd_p =return_tweet_words(df_p)
    dp= pd.DataFrame({'words': list(fd_p.keys()),'Count' : list(fd_p.values())})
    dp= dp.nlargest(columns = 'Count', n = 40)
    mask1=np.array(Image.open("photos\\happy_wc.png"))
    plot_wordcloud(data=dp,mask=mask1,contour_color='green').save(img1, format='PNG')
    happy='data:image/png;base64,{}'.format(base64.b64encode(img1.getvalue()).decode())
    
    img2=BytesIO()
    df_n=df1[df1['Overall_sentiment'] =='Negative']
    fd_n =return_tweet_words(df_n)
    dn= pd.DataFrame({'words': list(fd_n.keys()),'Count' : list(fd_n.values())})
    dn= dn.nlargest(columns = 'Count', n = 40)
    mask2=np.array(Image.open("photos\\sad_wc.png"))
    plot_wordcloud(data=dn,mask=mask2,contour_color='red').save(img2, format='PNG')
    sad='data:image/png;base64,{}'.format(base64.b64encode(img2.getvalue()).decode())
    
    img3=BytesIO()
    df_ne=df1[df1['Overall_sentiment'] =='Neutral']
    fd_ne =return_tweet_words(df_ne)
    dne= pd.DataFrame({'words': list(fd_ne.keys()),'Count' : list(fd_ne.values())})
    dne= dne.nlargest(columns = 'Count', n = 40)
    mask3=np.array(Image.open("photos\\neutral_wc.png"))
    plot_wordcloud(data=dne,mask=mask3,contour_color='white').save(img3, format='PNG')
    neutral='data:image/png;base64,{}'.format(base64.b64encode(img3.getvalue()).decode())
    
    return happy,neutral,sad


@app.callback(dash.dependencies.Output("Graph",'figure'),[dash.dependencies.Input('word_input','value')])
def update_word(h):
    def search(t):
        d1=str(t).count(h.lower())
        return d1
    
    df1=df[df["Country_code"]=='IN']
    df1["count"]=df1["Tweet_text"].apply(search)
    df2=df1[df1['count']>0]
    df3=df2.drop(['User_Id','Created_at','Country_code','Latitude','Longitude','Tweet_text',
                  'Sentiment_compound','Overall_sentiment'],axis=1)
    df3=df3.groupby('State').sum()
    df3.sort_values(by=['count'], inplace=True,ascending=False)
    df3=df3.rename_axis('index').reset_index()
    df3.rename(columns={"index":"state"})
    
    fig5=px.bar(df3.head(10),x='index',y='count',hover_data=['index','count'],
                color='count',color_continuous_scale=px.colors.sequential.Plasma_r,
                labels={'pop':'count'},height=400)
    fig5.update_traces(hovertemplate='State: %{x} <br>Count: %{y}')
    fig5.update_layout(
        xaxis_title="Top States",yaxis_title="Count",
        paper_bgcolor="black",font=dict(family="Nunito Sans",size=20,color="#DDDDDD"))
    
    return fig5

@app.callback(dd.Output('polar_bar_graph', 'figure'), [dd.Input('polar_bar_graph', 'id')])
def return_pie_chart(id):
    fd =return_tweet_words(df[df["Country_code"]=='IN'])
    d = pd.DataFrame({'Hashtag': list(fd.keys()),
                  'Count' : list(fd.values())})
    d = d.nlargest(columns = 'Count', n = 10)
    fig= px.bar_polar(d, r='Count', theta='Hashtag',hover_data=['Hashtag','Count'],
                   color='Hashtag', template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r)
    fig.update_traces(hovertemplate='Count: %{r}')
    
    return fig




app.config['suppress_callback_exceptions'] = True

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


H:\anaconda\lib\site-packages\ipykernel_launcher.py:594: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

